In [2]:
import sys
sys.path.append('/home/zhw272/programming/pipeline_utility')
sys.path.append('/home/zhw272/programming/pipeline_utility/src')
from src.sql_setup import session
from src.model.center_of_mass import CenterOfMass
from src.lib.sqlcontroller import SqlController
import notebooks.Will.experimental.get_coms_from_pickle as getcomp
import notebooks.Will.experimental.get_coms_from_database as getcomd
import notebooks.Will.experimental.get_transformed_coms as gettc
import numpy as np
from src.lib.comparison_tools import compare_lists

In [8]:
def query_brain_coms(brain, person_id=28, input_type_id=4,factor = np.array([0.325,0.325,20])):
    # default: person is bili, input_type is aligned
    rows = session.query(CenterOfMass)\
        .filter(CenterOfMass.active.is_(True))\
        .filter(CenterOfMass.prep_id == brain)\
        .filter(CenterOfMass.person_id == person_id)\
        .filter(CenterOfMass.input_type_id == input_type_id)\
        .all()
    row_dict = {}
    for row in rows:
        structure = row.structure.abbreviation
        row_dict[structure] = np.array([row.x, row.y, row.section])*factor
    return row_dict

In [11]:

def check_dict(d1:dict,d2:dict,verbose = True):
    if (d1.keys()==d2.keys()):
        bools = []
        for keyi in d1.keys():
            equal = np.all(np.isclose(d1[keyi],d2[keyi],atol = 1))
            if not equal and verbose:
                print('nonequal entry:')
                print(keyi,d1[keyi],d2[keyi])
            bools.append(equal)
            
    else:
        print('unequal keys')
        compare_lists(list(d1.keys()),list(d2.keys()))
        bools = None
    return np.all(bools)

In [27]:
#compare atlas
a1 = getcomp.get_atlas_com()
a2 = getcomd.get_atlas_com()
check_dict(a1,a2) 

True

In [28]:
#compare com save vs database corrected beth
prep_list = getcomp.get_prep_list()
for prepi in prep_list:
    d1 = getcomp.get_corrected_prepi_com(prepi)
    d2 = getcomd.get_corrected_prepi_com(prepi)
    print(prepi,check_dict(d1,d2) )

DK39 True
DK41 True
DK43 True
DK54 True
DK55 True
DK52 True


In [3]:
#compare com save vs database beth
prep_list = getcomp.get_prep_list()
for prepi in prep_list:
    d1 = getcomp.get_prepi_com(prepi)
    d2 = getcomd.get_prepi_com(prepi)
    print(prepi,check_dict(d1,d2) )

unequal keys
{'IC', '12N'}
in l2 not l1
DK39 None
DK41 True
DK43 True
unequal keys
{'IC', 'SC'}
in l2 not l1
DK54 None
unequal keys
in l1 not l2
{'6N_L', 'LC_L', '10N_L', '6N_R', '3N_L', '10N_R', '7N_R', '3N_R', '7N_L', 'Amb_R', 'DC_R', 'LC_R', 'Tz_L', 'Amb_L', 'Pn_R', 'LRt_L', 'Pn_L', '5N_L', 'LRt_R', 'DC_L', 'Tz_R', '5N_R'}
DK55 None
unequal keys
{'VLL_R', 'Pn_R', 'VCP_L', 'Pn_L', 'Amb_R', 'VCP_R', 'Sp5C_L', 'Sp5C_R', 'VLL_L'}
in l2 not l1
DK52 None


In [32]:
#compare com Com table(old) vs save beth
prep_list = getcomp.get_prep_list()
for prepi in prep_list:
    d1 = getcomp.get_prepi_com(prepi)
    d2 = query_brain_coms(prepi, person_id=2, input_type_id=1)
    print(prepi,check_dict(d1,d2) )

DK39 True
DK41 True
DK43 True
DK54 True
DK55 True
DK52 True


In [34]:
#compare com Com table(old) vs save corrected beth
prep_list = getcomp.get_prep_list()
prep_list.remove('DK55')
for prepi in prep_list:
    d1 = getcomp.get_corrected_prepi_com(prepi)
    d2 = query_brain_coms(prepi, person_id=2, input_type_id=2)
    print(prepi,check_dict(d1,d2) )


DK39 True
DK41 True
nonequal entry:
6N_R [12704.575  6462.17   4840.   ] [12704.575   245.05   4840.   ]
DK43 False
DK54 True
DK52 True


In [6]:
beth_aligned = gettc.get_beth_coms_aligned_to_atlas(getcomp.get_corrected_prepi_com)

In [14]:
prep_list

['DK39', 'DK41', 'DK43', 'DK54', 'DK55']

In [18]:
query_brain_coms(prepi, person_id=28, input_type_id=4,factor = np.array([10,10,20]))

{'7N_R': array([5163.63, 6145.24, 4328.86]),
 '7N_L': array([5441.91 , 6138.42 , 1893.556]),
 'Tz_R': array([4335.43, 6361.15, 3502.24]),
 'Tz_L': array([4381.43, 6362.29, 2557.28]),
 '5N_R': array([4607.46, 4995.83, 4384.06]),
 'Pn_L': array([3392.25, 6032.53, 2349.1 ]),
 'Pn_R': array([3488.1 , 6136.84, 3556.48]),
 '3N_L': array([3891.48, 3875.77, 2829.4 ]),
 '3N_R': array([3847.92, 3846.75, 3088.66]),
 '6N_R': array([5105.96, 5134.  , 3402.94]),
 '6N_L': array([5195.61, 5135.32, 2720.84]),
 '10N_L': array([6657.15, 5038.89, 2966.1 ]),
 '10N_R': array([6045.13 , 6029.78 , 1937.684]),
 'DC_L': array([5889.11 , 4566.51 , 1089.392]),
 'DC_R': array([5677.67, 4485.96, 5188.14]),
 'LC_R': array([5163.1 , 4279.56, 3868.18]),
 'LC_L': array([5164.15, 4255.9 , 2271.56]),
 'LRt_L': array([6782.75 , 6319.4  , 1984.086]),
 'LRt_R': array([6925.4 , 6375.3 , 4405.28]),
 '5N_L': array([4738.52 , 5042.69 , 1676.158])}

In [13]:
#compare com Com table(old) vs save corrected beth
prep_list = getcomp.get_prep_list()
prep_list.remove('DK52')
id = 0
for prepi in prep_list:
    d1 = beth_aligned[id]
    d2 = query_brain_coms(prepi, person_id=28, input_type_id=2,factor = np.array([10,10,20]))
    print(prepi,check_dict(d1,d2,verbose = False) )
    id+=1

DK39 True
DK41 True
DK43 False
DK54 True
unequal keys
in l1 not l2
{'Tz_L', 'Pn_L', 'Pn_R', '7N_R', 'LRt_L', '7N_L', '6N_R', 'Amb_L', 'LC_L', 'Amb_R', 'LC_R', 'Tz_R', 'LRt_R', '6N_L', '5N_L', '3N_R', 'DC_L', '3N_L', '10N_L', '10N_R', '5N_R', 'DC_R'}
DK55 None
